# Avance de proyecto 4: Sistema de recomendación
## Maestría en inteligencia artificial aplicada: Análisis de grandes volúmenes de datos


Equipo:
- Abraham Cabanzo Jimenez A01794355
- Ignacio Antonio Ruiz Guerra A00889972
- Moisés Díaz Malagón A01208580


En esta entrega presentamos la implementación final de los sistemas de recomendación así como la evaluación de desemepeño.

## Implementación final de sistemas de recomendación

Reescribimos los algoritmos contenidos en las entregas 1 y 2, utilizando programación orientada a objetos y estandarizando entradas y salidas entre ellos para facilitar su uso y para facilitar la evaluación.

In [ ]:
!pip install category_encoders

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from category_encoders.binary import BinaryEncoder

def preprocessing_algo1(df):
    df = df.copy()
    # Convertiremos gross de texto a cantidad numérica flotante
    df['Gross'] = df['Gross'].str.replace(',','').astype('float64')

    # Convertimos runtime de texto a cantidad numérica entera representando minutos
    df['Runtime'] = df['Runtime'].apply(lambda x: int(re.sub(r'\w*min$', '', x)))

    # Imputación de datos faltantes utilizando el promedio de las calificaciones
    imputer = SimpleImputer(strategy='mean')
    df['Meta_score'] = imputer.fit_transform(df['Meta_score'].values.reshape(-1, 1))

    # Imputación de datos faltantes utilizando la media de los ingresos recibidos,
    # pues la distribución de Gross tiene un sesgo fuerte a la derecha.
    imputer = SimpleImputer(strategy='median')
    df['Gross'] = imputer.fit_transform(df['Gross'].values.reshape(-1, 1))

    genre_encoder = BinaryEncoder()
    release_year_encoder = BinaryEncoder()
    director_encoder = BinaryEncoder()
    star1_encoder = BinaryEncoder()
    star2_encoder = BinaryEncoder()
    star3_encoder = BinaryEncoder()
    star4_encoder = BinaryEncoder()
    certificate_encoder = OneHotEncoder(drop='first', sparse_output=False)

    encoded_genre = genre_encoder.fit_transform(df['Genre'])
    encoded_director = director_encoder.fit_transform(df['Director'])
    encoded_star1 = star1_encoder.fit_transform(df['Star1'])
    encoded_star2 = star2_encoder.fit_transform(df['Star2'])
    encoded_star3 = star3_encoder.fit_transform(df['Star3'])
    encoded_star4 = star4_encoder.fit_transform(df['Star4'])

    encoded_release_year = release_year_encoder.fit_transform(df[['Released_Year']])
    encoded_release_year_column_names = release_year_encoder.get_feature_names_out(['Released_Year'])
    encoded_release_year = pd.DataFrame(encoded_release_year, columns=encoded_release_year_column_names, index=df.index)

    encoded_certificate = certificate_encoder.fit_transform(df[['Certificate']])
    encoded_certificate_column_names = certificate_encoder.get_feature_names_out(['Certificate'])
    encoded_certificate = pd.DataFrame(encoded_certificate, columns=encoded_certificate_column_names, index=df.index)


    final_df = df.copy()

    # Nótese que conservaremos las columnas originales pues para técnicas de NLP seguiremos utilizandolas
    final_df = pd.concat([final_df, encoded_genre, encoded_certificate, encoded_director, encoded_star1, encoded_star2, encoded_star3, encoded_star4, encoded_release_year], axis=1)

    columns = ['Series_Title', 'Genre', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']
    content = final_df[columns].copy()

    # concatenamos los textos, convertimos todo a minúsculas
    content['concatenated'] = content['Series_Title'] + " " + content['Genre'] + " " + content['Director'] + " " + content['Star1'] + " " + content['Star2'] + " " + content['Star3'] + " " + content['Star4']


    return content

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from utils import preprocessing_algo1
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process
from collections import Counter
import re


C:\Users\abrah\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# si se utiliza google colab
#from google.colab import drive
#import os
#drive.mount('/content/drive')
#os.chdir("drive/My Drive/")

In [ ]:
# BD para benchmark de primer algoritmo, basado en técnicas de NLP

#df = pd.read_csv("./imdb_top_1000.csv")
df = pd.read_csv("imdb_top_1000.csv")

# BD para resto de experimentos presentados en las entregas anteriores
ratings = pd.read_csv("ratings.dat", sep='::', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv("movies.dat", sep='::', names=['movie_id', 'movie_title', 'genres'], encoding='latin-1')

C:\Users\abrah\AppData\Local\Temp\ipykernel_17108\2068147516.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("ratings.dat", sep='::', names=['user_id', 'movie_id', 'rating', 'timestamp'])
C:\Users\abrah\AppData\Local\Temp\ipykernel_17108\2068147516.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv("movies.dat", sep='::', names=['movie_id', 'movie_title', 'genres'], encoding='latin-1')


In [ ]:
def global_recommender(top_n, movies, ratings):
    '''
    Recomendador global de películas.
    Hace recomendaciones generales con base en la popularidad de las películas, las más vistas. En este caso se considerarán como las mas vistas, las más veces calificadas.

    Parameters:
        top_n (int): Número de películas a recomendar.
        movies (pd.DataFrame): DataFrame con las películas.
        ratings (pd.DataFrame): DataFrame con las calificaciones.

    Returns:
        tupla: Título de las películas recomendadas, id de las películas recomendadas.
    '''
    movieid_to_title = dict(zip(movies['movie_id'], movies['movie_title'])) # mapeo de id de película a título de la película
    top_movies = ratings.value_counts('movie_id').sort_values(ascending=False).head(top_n) # calculamos vistas por película
    titles = [movieid_to_title[x] for x in top_movies.tolist()] # obtenemos los títulos de las películas más vistas
    movieids = top_movies.index.tolist() # obtenemos los ids de las películas más vistas
    return titles, movieids

In [ ]:
class CollaborativeFilterBasic():
    """
    Filtro colaborativo básico, basado en la matriz de usuario-item.
    No toma en cuenta las características de las películas.
    Funciona bajo la premisa de que a usuarios similares les interesan las mismas cosas.
    Aprende de los intereses de la población.

    """
    # matriz usuario-item, filas son usuarios, columnas son películas, contenido es la calificación bayesiana
    # es una matriz dispersa

    def __init__(self, movies, ratings):
        """
        Inicializa una instancia del filtro colaborativo básico.

        Parameters:
            movies (pd.DataFrame): DataFrame con las películas.
            ratings (pd.DataFrame): DataFrame con las calificaciones de las películas por usuario.

        Returns:
            None

        """

        self.movies = movies.copy()
        self.ratings = ratings.copy()

        # mapeo de títulos de películas a ids y viceversa
        self.title_to_movieid = dict(zip(self.movies['movie_title'], list(self.movies['movie_id'])))
        self.movieid_to_title = {v:k for k,v in self.title_to_movieid.items()}

        self.model = None

    def find_movieid(self, title):
        """
        Utiliza la librería fuzzywuzzy para encontrar el id de una película dado su título.
        Permite encontrar películas incluso si el título está mal escrito o no coincide exactamente.

        Parameters:
            title (str): Título de la película a buscar.

        Returns:
            int: Id de la película encontrada.
        """

        all_titles = self.movies['movie_title'].tolist()
        closest_match = process.extractOne(title, all_titles)
        return self.title_to_movieid[closest_match[0]]


    def __create_user_movie_matrix(self, df):
        """
        Crea la matriz usuario-película a partir de un DataFrame con las calificaciones de las películas por usuario.
        Guarda la matriz en disco para su uso posterior.

        Parameters:
            df (pd.DataFrame): DataFrame con las calificaciones de las películas por usuario.

        Returns:
            None
        """

        unique_users = df['user_id'].unique()
        unique_movies = df['movie_id'].unique()

        # mapeos de usuarios y películas a índices
        self.userid_to_idx = dict(zip(unique_users, list(range(len(unique_users)))))
        self.movieid_to_idx = dict(zip(unique_movies, list(range(len(unique_movies)))))

        # mapeos inversos
        self.idx_to_userid = {v:k for k,v in self.userid_to_idx.items()}
        self.idx_to_movieid = {v:k for k,v in self.movieid_to_idx.items()}

        users_idx = [self.userid_to_idx[id] for id in df['user_id']]
        movies_idx = [self.movieid_to_idx[id] for id in df['movie_id']]

        # generamos matriz dispersa
        self.user_movie_matrix = csr_matrix((df['rating'], (users_idx, movies_idx)), shape=(len(unique_users), len(unique_movies)))

        # guardamos matriz en disco
        save_npz('user_movie_matrix.npz', self.user_movie_matrix)

    def train(self, k=10, metric='cosine'):
        """
        Entrena el modelo de filtro colaborativo básico.

        Parameters:
            k (int): Número de vecinos más cercanos a considerar.
            metric (str): Medida de distancia a utilizar. Puede ser 'cosine' o 'euclidean'.

        Returns:
            None

        """

        # calculamos la matriz usuario-película
        self.__create_user_movie_matrix(self.ratings)

        # inicializamos el modelo de vecinos más cercanos
        self.model = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=metric)

        # entrenamos el modelo con la matriz usuario-película
        self.model.fit(self.user_movie_matrix.T)


    def recommend(self, movie_title=None, movie_id=None, k=10):
        """
        Hace recomendación de 10 películas similares a una dada.

        Parameters:
            movie_title (str): Título de la película a la que se le quiere encontrar películas similares.
            movie_id (int): Id de la película a la que se le quiere encontrar películas similares. Si no se proporciona, se buscará el id a partir del título, utilizando la librería fuzzywuzzy.
            k (int): Número de películas a recomendar.

        Returns:
            tupla: Título de las 10 películas recomendadas, id de las 10 películas recomendadas.
        """

        if movie_id is None:
            movie_id = self.find_movieid(movie_title)

        # encontramos los k vecinos más cercanos a la película dada
        distances, indices = self.model.kneighbors(self.user_movie_matrix.T[self.movieid_to_idx[movie_id]], n_neighbors=k+1)
        distances = distances.squeeze()
        indices = indices.squeeze()
        movie_list = [self.movieid_to_title[self.idx_to_movieid[idx]] for idx in indices[1:]]
        movies_ids = [self.idx_to_movieid[idx] for idx in indices[1:]]

        return movie_list, movies_ids

In [ ]:
class ContentBasedBasic():
    '''
    Filtro basado en contenido. No toma en cuenta a los usuarios, sino las características de las películas, pero a diferencia del primer algoritmo, este no utiliza técnicas de NLP.

    Soporta arranque en frio.
    '''

    def __init__(self, movies):
        """
        Inicializa una instancia del filtro basado en contenido.

        Parameters:
            movies (pd.DataFrame): DataFrame con las películas.

        Returns:
            None

        """
        self.movies = movies.copy()
        self.similarity_matrix = None

        # mapeos de títulos, ids de películas e índices, para facilitar la búsqueda
        self.title_to_idx = dict(zip(self.movies['movie_title'], list(self.movies.index)))
        self.title_to_movieid = dict(zip(self.movies['movie_title'], list(self.movies['movie_id'])))
        self.movieid_to_idx = dict(zip(self.movies['movie_id'], list(self.movies.index)))
        self.idx_to_title = {v:k for k,v in self.title_to_idx.items()}

    def find_movie_idx(self, title):
        """
        Utiliza la librería fuzzywuzzy para encontrar el índice de una película dado su título.
        Permite encontrar películas incluso si el título está mal escrito o no coincide exactamente.

        Parameters:
            title (str): Título de la película a buscar.

        Returns:
            int: índice de la película encontrada, dentro del DataFrame de películas.
        """

        all_titles = self.movies['movie_title'].tolist()
        closest_match = process.extractOne(title, all_titles)
        return self.title_to_idx[closest_match[0]]

    @staticmethod
    def __calc_similarity_matrix(movies):
        """
        Calcula la matriz de similitud entre las películas.

        Parameters:
            movies (pd.DataFrame): DataFrame con las películas.

        Returns:
            np.array: Matriz de similitud entre las películas.
        """

        # generamos un array de géneros de las películas
        movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

        # eliminamos las películas que no tienen género
        genre_counter = Counter(g for genres in movies['genres'] for g in genres)
        movies = movies[movies['genres']!='(no genres listed)']
        del genre_counter['(no genres listed)']

        # limpiamos los títulos de las películas, para eliminar el año de publicación
        movies['clean_title'] = movies['movie_title'].apply(lambda x: re.sub(r'\s*\([^)]*\)', '', x))

        # extraemos el año de publicación de las películas y la década
        movies['year'] = movies['movie_title'].apply(ContentBasedBasic.__extract_year)
        movies['decade'] = movies['year'].apply(lambda x: x - x%10)

        # A continuación transformamos a one hot encoding los generos para generar un vector de 1s y 0s para cada película dependiendo si pertenece o no a un género en particular
        genres = list(genre_counter.keys())
        for gender in genres:
            movies[gender] = movies['genres'].transform(lambda x: int(gender in x))

        # generamos one-hot encodings para las décadas
        movie_decades = pd.get_dummies(movies['decade'])

        # concatenamos las características de las películas
        movie_features = pd.concat([movies[genres], movie_decades], axis=1)

        # calculamos la similitud de coseno entre las características de las películas
        similarity_matrix = cosine_similarity(movie_features, movie_features)
        return similarity_matrix

    @staticmethod
    def __extract_year(title):
        """
        Función que extrae el año de las películas del título utilizando expresiones regulares.

        Parameters:
            title (str): Título de la película.

        Returns:
            int: Año de la película
        """
        year = re.search(r'\((\d{4})\)', title)
        return int(year.group(1)) if year else None

    def train(self):
        """
        Entrena el modelo de filtro basado en contenido.
        Calcula la matriz de similaridad.

        Returns:
            None
        """
        self.similarity_matrix = self.__calc_similarity_matrix(self.movies)


    def recommend(self, movie_title=None, movie_id=None, top_n=10):
        """
        Hace recomendación de películas similares a una dada basandose en el contenido de las películas.

        Parameters:
            movie_title (str): Título de la película a la que se le quiere encontrar películas similares.
            movie_id (int): Id de la película a la que se le quiere encontrar películas similares. Si no se proporciona, se buscará el id a partir del título, utilizando la librería fuzzywuzzy.
            top_n (int): Número de películas a recomendar.

        Returns:
            tupla: Título de las películas recomendadas, id de las películas recomendadas.
        """

        if movie_id:
            movie_idx = self.movieid_to_idx[movie_id]
        elif movie_title:
            movie_idx = self.find_movie_idx(movie_title)

        # encontramos las películas más similares a la dada
        sim_scores = list(enumerate(self.similarity_matrix[movie_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:(top_n+1)] # seleccionamos las top_n películas más similares

        similar_movies_idx = [i[0] for i in sim_scores]
        similar_movies = [self.idx_to_title[x] for x in similar_movies_idx]
        similar_moviesids = [self.title_to_movieid[x] for x in similar_movies]

        return similar_movies, similar_moviesids


In [ ]:
!pip install pytorch_lightning

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl

class RatingsDataset(Dataset):
    '''
    Dataset de Pytorch para generar los datos de entrenamiento para el modelo de deep learning.
    Toma como entrada los ratings de las peliculas y transforma las calificaciones existentes en etiquetas de que el usuario vio la pelicula y genera N ejemplos no existentes para etiquetas de que el usuario no vio la pelicula. Esto servirá para entrenar el modelo de deep learning.

    '''
    def __init__(self, ratings, negative_examples_qty, all_movieids):
        """
        Inicializa una instancia del dataset de ratings.

        Parameters:
            ratings (pd.DataFrame): DataFrame con las interacciones de usuarios.
            negative_examples_qty (int): Cantidad de ejemplos negativos a generar.
            all_movieids (list): Lista con los IDs de todas las películas.

        Returns:
            None
        """

        self.all_movieids = all_movieids
        self.negative_examples_qty = negative_examples_qty
        self.users, self.movies, self.labels = self._get_dataset(ratings, self.all_movieids)

    def __len__(self):
        """
        Retorna la cantidad de usuarios en el dataset

        Returns:
            int: Cantidad de usuarios en el dataset.
        """
        return len(self.users)

    def __getitem__(self, idx):
        """
        Retorna un ejemplo del dataset de entrenamiento.

        Parameters:
            idx (int): Índice del ejemplo a retornar.

        Returns:
            tupla: id de usuario, id de película, etiqueta (visto o no visto).
        """
        return self.users[idx], self.movies[idx], self.labels[idx]

    def _get_dataset(self, ratings, all_movieids):
        """
        Genera el dataset de entrenamiento para el modelo de deep learning.

        Parameters:
            ratings (pd.DataFrame): DataFrame con las interacciones de usuarios.
            all_movieids (list): Lista con los IDs de todas las películas.

        Returns:
            tupla: Lista con los IDs de los usuarios, lista con los IDs de las películas, lista con las etiquetas (película vista o no vista).
        """
        users, movies, labels = [], [], []

        # hacemos un set de pares, usuario - pelicula, no existen pares duplicados
        user_movie_set = set(zip(ratings['user_id'], ratings['movie_id']))

        for (user_id, movie_id) in user_movie_set:
            users.append(user_id)
            movies.append(movie_id)
            labels.append(1) # consideramos que si calificó la película, la vio
            for idx in range(self.negative_examples_qty):
                # selecciona una pelicula aleatoria
                negative_item = np.random.choice(all_movieids)
                # revisar que el usuario no haya interactuado con esta pelicula antes
                while (user_id, negative_item) in user_movie_set:
                    negative_item = np.random.choice(all_movieids)
                users.append(user_id)
                movies.append(negative_item)
                labels.append(0) # no vio la pelicula

        return users, movies, labels

class DLRecommenderModel(pl.LightningModule):
    '''
    Sistema de recomendación que utiliza un modelo de deep learning para implementar un filtro colaborativo que modela una función de similaridad entre vectores (embeddings) de usuarios y películas.
    '''

    _EMBEDDING_DIMENSIONS = 6 # dimensiones de los embeddings

    def __init__(self, n_users, n_movies, ratings, all_movieids, negative_examples_qty):
        """
        Inicializa una instancia del modelo de deep learning.

        Parameters:
            n_users (int): Cantidad de usuarios.
            n_movies (int): Cantidad de películas.
            ratings (pd.DataFrame): DataFrame con las calificaciones de las películas por usuario.
            all_movieids (list): Lista con los IDs de todas las películas.
            negative_examples_qty (int): Cantidad de ejemplos negativos a generar.

        Returns:
            None
        """

        super().__init__() # llamamos al constructor de la clase padre
        self.negative_examples_qty = negative_examples_qty
        self.user_embedding = nn.Embedding(num_embeddings=n_users, embedding_dim=self._EMBEDDING_DIMENSIONS) # usamos capa Embedding de pytorch
        self.movie_embedding = nn.Embedding(num_embeddings=n_movies, embedding_dim=self._EMBEDDING_DIMENSIONS)

        # posterior a las capas embeddings se aplican dos capas densas, también llamadas Full connected.
        self.fc1 = nn.Linear(12, 64) # 12 features de entrada = 6 (de user embedding) + 6 (de movie embedding), 64 features de salida o neuronas
        self.fc2 = nn.Linear(64, 32) # 64 features de entrada, 32 features de salida
        self.fc3 = nn.Linear(32, 1) # 32 features de entrada, 1 feature de salida (prediccion)
        self.ratings = ratings.copy()
        self.all_movieids = all_movieids.copy()


    def forward(self, user_id, movie_id):
        """
        Hace una pasada hacia adelante en el modelo. Tanto para entrenamiento compara inferencia.

        Parameters:
            user_id (int): ID del usuario.
            movie_id (int): ID de la película.

        Returns:
            float: porcentaje de probabilidad de que el usuario haya visto la película.
        """

        # calculamos los embeddings
        user_embedding = self.user_embedding(user_id)
        movie_embedding = self.movie_embedding(movie_id)
        # concatenamos los embeddings de usuario y pelicula
        x = torch.cat([user_embedding, movie_embedding], dim=1)

        # aplicamos las capas densas
        x = nn.ReLU()(self.fc1(x)) # función de activación ReLU para capas intermedias
        x = nn.ReLU()(self.fc2(x))
        out = nn.Sigmoid()(self.fc3(x)) # función de activación sigmoide para obtener un valor entre 0 y 1 como salida final

        return out


    def training_step(self, batch):
        """
        Realiza un paso de entrenamiento en el modelo.

        Parameters:
            batch (tupla): conjunto de datos de entrenamiento.
        Returns:
            float: Pérdida en el paso de entrenamiento.
        """
        user_ids, movie_ids, labels = batch
        predictions = self(user_ids, movie_ids)
        # utilizaremos la función de pérdida Binary Cross Entropy, pues nuestro problema de clasificación es de tipo binario (dos clases)
        loss = nn.BCELoss()(predictions, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        """
        Configura el optimizador para el modelo.

        Returns:
            torch.optim.Optimizer: Optimizador a utilizar.
        """

        # utilizamos Adam como optimizador, se puede ajustar el learning rate
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        """
        Genera el DataLoader para el entrenamiento del modelo.

        Returns:
            DataLoader: DataLoader para el entrenamiento del modelo.
        """

        dataset = RatingsDataset(self.ratings, self.negative_examples_qty, self.all_movieids)
        return DataLoader(dataset, batch_size=512, num_workers=0)

class CollaborativeFilterDL():
    """
    Algoritmo avanzado. Filtro colaborativo basado en deep learning.
    Utiliza retroalimentación implicita. Es decir, todas las peliculas que un usuario califica se consideran como vistas.

    Durante el entrenamiento se calculan los embeddings tanto de los usuarios como de las películas así como los pesos de las redes neuronales que modelan la función de similaridad entre los embeddings.
    Representa un problema de clasificación binaria, donde la etiqueta es 1 si el usuario vio la película y 0 si no la vio. Requiere ser entrenado y actualizado para nuevos usuarios y películas.
    """

    def __init__(self, ratings, movies):
        """
        Inicializa una instancia del filtro colaborativo basado en deep learning.

        Parameters:
            ratings (pd.DataFrame): DataFrame con las calificaciones de las películas por usuario.
            movies (pd.DataFrame): DataFrame con las películas.

        Returns:
            None
        """

        self.ratings = ratings.copy()
        self.movies = movies.copy()

        # preparamos los datos, generamos los ratings implícitos
        self.__prepare_data()

        self.model = None

        # mapeos de títulos con ids de películas, para facilitar la búsqueda
        self.title_to_movieid = dict(zip(self.movies['movie_title'], list(self.movies['movie_id'])))
        self.movieid_to_title = {v:k for k,v in self.title_to_movieid.items()}

    def find_movieid(self, title):
        """
        Utiliza la librería fuzzywuzzy para encontrar el id de una película dado su título.
        Permite encontrar películas incluso si el título está mal escrito o no coincide exactamente.

        Parameters:
            title (str): Título de la película a buscar.

        Returns:
            int: Id de la película encontrada.
        """

        all_titles = self.movies['movie_title'].tolist()
        closest_match = process.extractOne(title, all_titles)
        return self.title_to_movieid[closest_match[0]]


    def __prepare_data(self):
        """
        Prepara los datos para el entrenamiento del modelo de deep learning.
        De las interacciones de usuarios considera la última como parte del test set y el resto como parte del train set.
        Implementa retroalimentación implícita.

        Returns:
            None
        """
        # utilizaremos los ultimos ejemplos de cada usuario como parte del test set
        self.ratings['latest_rank'] = self.ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
        train_ratings = self.ratings[self.ratings['latest_rank'] != 1]
        test_ratings = self.ratings[self.ratings['latest_rank'] == 1]

        # conservamos unicamente user_id, movie_id, y rating
        self.train_ratings = train_ratings[['user_id', 'movie_id', 'rating']]
        self.test_ratings = test_ratings[['user_id', 'movie_id', 'rating']]

        # Retroalimentación implícita: a todos los ratings se les considera como vistos por el usuario
        self.train_ratings.loc[:, 'rating'] = 1

    def train(self):
        """
        Entrena el modelo de filtro colaborativo basado en deep learning.

        Returns:
            None

        """

        num_users = self.ratings['user_id'].max() + 1
        num_items = ratings['movie_id'].max() + 1
        self.all_movieids = ratings['movie_id'].unique()

        # inicializamos el modelo
        self.model = DLRecommenderModel(num_users, num_items, self.train_ratings, self.all_movieids, negative_examples_qty=5)

        # entrenamos el modelo
        trainer = pl.Trainer(max_epochs=6, logger=False, accelerator='cpu', devices=1) # se puede ajustar el número de épocas, y acelerador si se tiene GPU
        trainer.fit(self.model)
        return self.model

    def recommend(self, user_id, model):
        """
        Hace 10 recomendaciones de películas para un usuario dado.

        Parameters:
            user_id (int): ID del usuario.
            model (DLRecommenderModel): Modelo de deep learning entrenado.

        Returns:
            list: Lista con los 10 títulos de las películas recomendadas.
        """
        # obtenemos las películas que el usuario no ha visto
        not_seen_movies = self.ratings[self.ratings['user_id'] == user_id]['movie_id'].unique()

        # hacemos inferencia con el modelo para obtener los scores de las películas no vistas
        not_seen_scores = np.squeeze(model(torch.tensor([user_id]*len(not_seen_movies)), torch.tensor(not_seen_movies)).detach().numpy())

        # ordenamos las películas por score y regresamos las 10 más altas
        not_seen_pairs = zip(not_seen_movies, not_seen_scores)
        not_seen_pairs = sorted(not_seen_pairs, key=lambda x: x[1], reverse=True)
        top_recommended = not_seen_pairs[1:11]
        top_recommended = [self.movieid_to_title[x[0]] for x in top_recommended]

        return top_recommended


In [ ]:

class ContentBasedNLP():
    '''
    Sistema de recomendación basado en contenido.
    Técnicas de NLP, en específico TF-IDF y similaridad coseno.
    '''

    def __init__(self, content):
        nltk.download('stopwords')
        self.stopwords = stopwords.words('english')
        self.content = content
        self.__generate_similarity_mtrx()

    def __generate_sentences(self, doc):
        '''
        El preprocesamiento a utilizar consiste en conservar únicamente caracteres alfabéticos, eliminando signos de puntuación, caracteres especiales y números.
        Solamente consideremos palabras (tokens) con longitud mayor a un caracter.
        Convertiremos todo a minúsculas.
        Eliminaremos del texto los stopwords del diccionario de stop words de la librería NLTK en inglés.

        Parameters:
            doc (str): oración a procesar.

        Returns:
            str: oración procesada.
        '''
        words = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]', ' ', doc)
        words = re.sub(r'\s{2,}', ' ', words.strip())
        words = words.lower().split()
        tokens = [ w for w in words if w not in self.stopwords]

        return " ".join(tokens)

    def __generate_similarity_mtrx(self):
        """
        Genera la matriz de similaridad entre las películas.
        Utiliza TF-IDF y similaridad coseno.

        Returns:
            None
        """
        sentences = [self.__generate_sentences(x) for x in self.content['concatenated']]
        tfidfvectorizer = TfidfVectorizer()
        tfidf_matrix = tfidfvectorizer.fit_transform(sentences)
        self.similarity_mtrx = linear_kernel(tfidf_matrix, tfidf_matrix)
        self.movies_index = pd.Series(self.content.index, index=self.content['Series_Title']).drop_duplicates()

    def recommend(self, movie_name):
        """
        Hace recomendación de 10 películas similares a una dada basandose en el contenido de las películas.

        Parameters:
            movie_name (str): Título de la película a la que se le quiere encontrar películas similares.

        Returns:
            list: Título de las 10 películas recomendadas.
        """

        idx = self.movies_index['Star Wars']
        # obtenemos el vector de similitudes de esa pelicula con todas las otras
        similarity_scores = list(enumerate(self.similarity_mtrx[idx]))
        # ordenamos las pel[iculas con base en los scores de similaidad
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        # podemos observar las 10 películas más similares, omitimos la primera pues es Star Wars
        similarity_scores = similarity_scores[1:11]
        title_idxs = [i[0] for i in similarity_scores]
        return list(self.content['Series_Title'].iloc[title_idxs].values)

In [ ]:
content_algo1 = preprocessing_algo1(df)
content_nlp = ContentBasedNLP(content_algo1)

collaborative_basic = CollaborativeFilterBasic(movies, ratings)

content_basic = ContentBasedBasic(movies)

collaborative_deeplearning = CollaborativeFilterDL(ratings, movies)

# entrenamos los algoritmos, algunos son entrenamientos no supervisados y otros supervisados
collaborative_basic.train()
content_basic.train()
trained_model = collaborative_deeplearning.train() # almacenamos el modelo entrenado

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abrah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\abrah\anaconda3\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory C:\Users\abrah\checkpoints exists and is not empty.

  | Name            | Type      | Params | Mode 
------------------------------------------------------
0 | user_embedding  | Embedding | 36.2 K | train
1 | movie_embedding | Embedding | 23.7 K | train
2 | fc1             | Linear    | 832    | train
3 | fc2             | Linear    | 2.1 K  | train
4 | fc3             | Linear    | 33     | train
------------------------------------------------------
62.9 K    Trainable params
0         Non-trainable params
62.9 K    Total params
0.252     Total estimated model params size (MB)
C:\Users\abrah\anaconda3\

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=6` reached.


## Pruebas de rendimiento

Se realizó la prueba de rendimiento a los algoritmos que se basan en el dataset de MovieLens, pues se requiere de un set de pruebas etiquetado para determinar si el algoritmo recomienda peliculas que en efecto el usuario ha visto.

Se utilizará el métrico **HitRatio@10** que indica la proporción de interacciones en las cuales el algoritmo ha recomendado al menos un elemento correcto (que el usuario ha visto) dentro del top 10 de elementos recomendados.

También se cuenta con la función **PrecisionRatio@10**, que evalúa la precisión de los diferentes modelos, al calcular la proporción de elementos relevantes en los 10 mejores resultados para cada interacción usuario-artículo.

La tercer métrica para la evaluación de estos modelos es **Mean Reciprocal Rank (MRR)**, la cuál se usa para evaluar la calidad de las recomendaciones en un sistema de recomendación. Se define como el recíproco del rango de la primera recomendación relevante. Si no hay una recomendación relevante, la puntuación es 0.


Tomaremos el set de datos de prueba calculado para el filtro colaborativo basado en deep learning y las interacciones de la base de datos de ratings.

In [ ]:
test_set = collaborative_deeplearning.test_ratings
all_movieids = collaborative_deeplearning.all_movieids
user_interactions = set(zip(test_set['user_id'], test_set['movie_id']))
user_to_seen_movies_dict = ratings.groupby('user_id')['movie_id'].apply(list).to_dict()

## HitRatio@10


$$
\text{Hit Ratio@k} = \frac{\text{Number of hits in top-k recommendations}}{\text{Total number of users}}
$$




In [ ]:
from tqdm.notebook import tqdm  # Importa la barra de progreso

def calc_hit_ratio(model_name='collaborative_basic'):
    hits = []  # Lista para almacenar si la película está en el top 10 recomendado
    i = 0  # Inicializa el contador para iterar sobre interacciones de usuario

    # Itera sobre las primeras 100 interacciones de usuario
    for (user_id, movie_id) in tqdm(list(user_interactions)[0:100]):
        seen_movies = user_to_seen_movies_dict[user_id]  # Películas vistas por el usuario

        if model_name == 'collaborative_deeplearning':
            # Películas no vistas
            non_seen_movies = set(all_movieids) - set(seen_movies)
            # Selecciona aleatoriamente películas no vistas y agrega la película actual
            selected_movies = list(np.random.choice(list(non_seen_movies), 99))
            test_movies = selected_movies + [movie_id]
            # Predice las etiquetas para las películas seleccionadas
            predicted_labels = np.squeeze(trained_model(torch.tensor([user_id]*100), torch.tensor(test_movies)).detach().numpy())

            # Obtiene los índices de las películas más recomendadas
            top_10_idx = [test_movies[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()] # i puede ir de 0 a 100, convertimos a indice de pelicula

        elif model_name == 'collaborative_basic':
            # Utiliza el modelo colaborativo básico para recomendar películas basadas en las películas vistas por el usuario
            for seen_movie in seen_movies:
                _, top_10_idx = collaborative_basic.recommend(movie_id=seen_movie)
                if movie_id in top_10_idx:
                    break

        elif model_name == 'content_basic':
            # Utiliza el modelo de contenido básico para recomendar películas basadas en las películas vistas por el usuario
            for seen_movie in seen_movies:
                _, top_10_idx = content_basic.recommend(movie_id=seen_movie)
                if movie_id in top_10_idx:
                    break

        elif model_name == 'global':
            # Utiliza un recomendador global para recomendar películas basadas en las calificaciones y películas disponibles
            test_movie = np.random.choice(list(set(seen_movies) - set([movie_id])))
            _, top_10_idx = global_recommender(top_n=10, movies=movies, ratings=ratings)

        # Verifica si la película actual está en el top 10 recomendado
        if movie_id in top_10_idx:
            hits.append(1)  # La película está en el top 10
        else:
            hits.append(0)  # La película no está en el top 10

    return np.mean(hits)*100  # Calcula el porcentaje de éxitos


In [ ]:
print(f"Hit ratio global: {calc_hit_ratio('global'):.2f}%")
print(f"Hit ratio collaborative_basic: {calc_hit_ratio('collaborative_basic'):.2f}%")
print(f"Hit ratio content_basic: {calc_hit_ratio('content_basic'):.2f}%", )
print(f"Hit ratio collaborative_deeplearning: {calc_hit_ratio('collaborative_deeplearning'):.2f}%")


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio global: 0.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio collaborative_basic: 57.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio content_basic: 52.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Hit ratio collaborative_deeplearning: 48.00%



Estas métricas indican claramente que los modelos personalizados son superiores al modelo global en términos de hit ratio,
y dentro de los modelos personalizados, los más simples (colaborativo básico,57%)
pueden ser más efectivos que los modelos más complejos (deep learning, 48%) en ciertas circunstancias.


## Precision@10


$$
\text{Precision@k} = \frac{\text{Number of relevant items in top-k recommendations}}{k}
$$


In [ ]:
def precision_at_k(model_name='collaborative_basic', k=10):
    precision_values = []  # Lista para almacenar los valores de precisión
    # Itera sobre las primeras 100 interacciones de usuario
    for (user_id, movie_id) in tqdm(list(user_interactions)[0:100]):
        seen_movies = user_to_seen_movies_dict[user_id]  # Películas vistas por el usuario

        if model_name == 'collaborative_deeplearning':
            # Películas no vistas
            non_seen_movies = set(all_movieids) - set(seen_movies)
            # Selecciona aleatoriamente películas no vistas y agrega la película actual
            selected_movies = list(np.random.choice(list(non_seen_movies), 99))
            test_movies = selected_movies + [movie_id]
            # Predice las etiquetas para las películas seleccionadas
            predicted_labels = np.squeeze(trained_model(torch.tensor([user_id]*100), torch.tensor(test_movies)).detach().numpy())

            # Obtiene los índices de las películas más recomendadas hasta el k-ésimo
            top_k_idx = [test_movies[i] for i in np.argsort(predicted_labels)[::-1][0:k].tolist()] # i puede ir de 0 a 100, convertimos a indice de pelicula

        elif model_name == 'collaborative_basic':
            # Utiliza el modelo colaborativo básico para recomendar películas basadas en las películas vistas por el usuario
            for seen_movie in seen_movies:
                _, top_k_idx = collaborative_basic.recommend(movie_id=seen_movie, k=k)
                if movie_id in top_k_idx:
                    break

        elif model_name == 'content_basic':
            # Utiliza el modelo de contenido básico para recomendar películas basadas en las películas vistas por el usuario
            for seen_movie in seen_movies:
                _, top_k_idx = content_basic.recommend(movie_id=seen_movie, top_n=k)
                if movie_id in top_k_idx:
                    break

        elif model_name == 'global':
            # Utiliza un recomendador global para recomendar películas basadas en las calificaciones y películas disponibles
            test_movie = np.random.choice(list(set(seen_movies) - set([movie_id])))
            _, top_k_idx = global_recommender(top_n=k, movies=movies, ratings=ratings)

        # Verifica si la película actual está en los k primeros recomendados
        if movie_id in top_k_idx:
            precision_values.append(1)  # La película está en los k primeros
        else:
            precision_values.append(0)  # La película no está en los k primeros

    return np.mean(precision_values)*100  # Calcula el porcentaje de precisión


In [ ]:

print(f"Precision ratio global: {precision_at_k('global'):.2f}%")
print(f"Precision ratio collaborative_basic: {precision_at_k('collaborative_basic'):.2f}%")
print(f"Precision ratio content_basic: {precision_at_k('content_basic'):.2f}%", )
print(f"Precision ratio collaborative_deeplearning: {precision_at_k('collaborative_deeplearning'):.2f}%")


  0%|          | 0/100 [00:00<?, ?it/s]

Precision ratio global: 0.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Precision ratio collaborative_basic: 57.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Precision ratio content_basic: 52.00%


  0%|          | 0/100 [00:00<?, ?it/s]

Precision ratio collaborative_deeplearning: 49.00%




Precision ratio global: Con un valor del 0.00% que nos indica que no pudo recomendar nada.

Precision ratio collaborative_basic: Alcanzando un 57.00%, este modelo tuvo el mejor desempeño en términos de precisión dentro de los 10 primeros elementos.

Precision ratio content_basic: Con un valor del 52.00%, el modelo de contenido básico también logró un rendimiento decente en cuanto a la precisión de las recomendaciones dentro de los 10 primeros elementos.

Precision ratio collaborative_deeplearning: Este modelo tuvo un desempeño ligeramente inferior, con un 49.00% de precisión.

## Mean Reciprocal Rank (MRR)

$$
\text{MRR} = \frac{1}{\text{Total number of users}} \sum_{i=1}^{\text{Total number of users}} \frac{1}{\text{Rank of first relevant item for user } i}
$$



In [ ]:
from joblib import Parallel, delayed  # Importa la función Parallel y delayed de joblib

def calc_mean_reciprocal_rank_parallel(model_name='collaborative_basic'):
    def calculate_rr(user_id, movie_id):
        seen_movies = user_to_seen_movies_dict[user_id]  # Películas vistas por el usuario

        if model_name == 'collaborative_deeplearning':
            # Películas no vistas
            non_seen_movies = set(all_movieids) - set(seen_movies)
            # Selecciona aleatoriamente películas no vistas y agrega la película actual
            selected_movies = list(np.random.choice(list(non_seen_movies), 99))
            test_movies = selected_movies + [movie_id]
            # Predice las etiquetas para las películas seleccionadas
            predicted_labels = np.squeeze(trained_model(torch.tensor([user_id]*100), torch.tensor(test_movies)).detach().numpy())
            # Obtiene los índices de las películas más recomendadas
            top_10_idx = [test_movies[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]

        elif model_name == 'collaborative_basic':
            # Utiliza el modelo colaborativo básico para recomendar películas basadas en las películas vistas por el usuario
            for seen_movie in seen_movies:
                _, top_10_idx = collaborative_basic.recommend(movie_id=seen_movie)
                if movie_id in top_10_idx:
                    break

        elif model_name == 'content_basic':
            # Utiliza el modelo de contenido básico para recomendar películas basadas en las películas vistas por el usuario
            for seen_movie in seen_movies:
                _, top_10_idx = content_basic.recommend(movie_id=seen_movie)
                if movie_id in top_10_idx:
                    break

        elif model_name == 'global':
            # Utiliza un recomendador global para recomendar películas basadas en las calificaciones y películas disponibles
            _, top_10_idx = global_recommender(top_n=10, movies=movies, ratings=ratings)

        # Calcula el rango de la película actual y su recíproco
        rank = top_10_idx.index(movie_id) + 1 if movie_id in top_10_idx else 0
        reciprocal_rank = 1 / rank if rank != 0 else 0
        return reciprocal_rank

    # Calcula los recíprocos de rango de forma paralela para todas las interacciones de usuario
    reciprocal_ranks = Parallel(n_jobs=-1)(delayed(calculate_rr)(user_id, movie_id) for (user_id, movie_id) in tqdm(list(user_interactions)))
    return np.mean(reciprocal_ranks)  # Calcula la media de los recíprocos de rango


In [ ]:
print(f"Mean Reciprocal Rank (MRR) for global model: {calc_mean_reciprocal_rank_parallel('global'):.4f}")
print(f"Mean Reciprocal Rank (MRR) for collaborative_basic model: {calc_mean_reciprocal_rank_parallel('collaborative_basic'):.4f}")
print(f"Mean Reciprocal Rank (MRR) for content_basic model: {calc_mean_reciprocal_rank_parallel('content_basic'):.4f}")
print(f"Mean Reciprocal Rank (MRR) for collaborative_deeplearning model: {calc_mean_reciprocal_rank_parallel('collaborative_deeplearning'):.4f}")

  0%|          | 0/6040 [00:00<?, ?it/s]

Mean Reciprocal Rank (MRR) for global model: 0.0092


  0%|          | 0/6040 [00:00<?, ?it/s]

Mean Reciprocal Rank (MRR) for collaborative_basic model: 0.1883


  0%|          | 0/6040 [00:00<?, ?it/s]

Mean Reciprocal Rank (MRR) for content_basic model: 0.1837


  0%|          | 0/6040 [00:00<?, ?it/s]

Mean Reciprocal Rank (MRR) for collaborative_deeplearning model: 0.1888


Se observa que bajo esta métrica, el mejor modelo resulta ser el de deep learning, con una puntuación del 18.88%. Muy de cerca queda el del filtro colaborativo con 18.83%.